### In this notebook:
Two LSTM combined together to classify MJO-ENSO and ERA5 timeseries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('./drive/My Drive/Tropical_Cyclones_Thesis/ERA5_Dataset')

In [3]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from keras.models import load_model


import pickle
import metrics
import datashift

In [4]:
import tensorflow as tf
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.12.0


 # Loading Datasets

In [5]:
df_era5 = pd.read_csv('./DATASET_COMPLETED/ensemble_dataset/ERA5_16zones_avg_std_10D.csv')
df_era5 = df_era5.set_index('DATE')
df_era5 = df_era5.drop(columns=['S.IndAll', 'S.IndGen'])
df_era5 = df_era5[df_era5.index > '1980-02']
df_era5 = df_era5[df_era5.index < '2022']

# Create MinMaxScaler instance
scaler = MinMaxScaler()
# Fit the scaler on the reshaped dataset and transform it
data_scaled = scaler.fit_transform(df_era5)
df_era5_scaled = pd.DataFrame(data_scaled, columns=df_era5.columns)
df_era5_scaled.index = df_era5.index
df_era5_scaled

,P_Mean_Z1-0,P_Mean_Z1-1,P_Mean_Z1-2,P_Mean_Z1-3,P_Mean_Z1-4,P_Mean_Z1-5,P_Mean_Z1-6,P_Mean_Z1-7,P_Mean_Z1-8,P_Mean_Z1-9,...,Air_Density_Std_Z16-1,Air_Density_Std_Z16-2,Air_Density_Std_Z16-3,Air_Density_Std_Z16-4,Air_Density_Std_Z16-5,Air_Density_Std_Z16-6,Air_Density_Std_Z16-7,Air_Density_Std_Z16-8,Air_Density_Std_Z16-9,Air_Density_Std_Z16-10
DATE,,,,,,,,,,,,,,,,,,,,,
1980-02-01,0.400255,0.364918,0.382609,0.416793,0.386992,0.370588,0.361763,0.434279,0.449898,0.489865,...,0.318328,0.273482,0.336515,0.403900,0.353448,0.402396,0.425584,0.422273,0.409480,0.392453
1980-02-02,0.412313,0.400255,0.364918,0.382609,0.416793,0.386992,0.370588,0.361763,0.434279,0.449898,...,0.376670,0.318328,0.273482,0.336515,0.403900,0.353448,0.402396,0.425584,0.422273,0.409480
1980-02-03,0.376618,0.412313,0.400255,0.364918,0.382609,0.416793,0.386992,0.370588,0.361763,0.434279,...,0.404121,0.376670,0.318328,0.273482,0.336515,0.403900,0.353448,0.402396,0.425584,0.422273
1980-02-04,0.348375,0.376618,0.412313,0.400255,0.364918,0.382609,0.416793,0.386992,0.370588,0.361763,...,0.363910,0.404121,0.376670,0.318328,0.273482,0.336515,0.403900,0.353448,0.402396,0.425584
1980-02-05,0.286090,0.348375,0.376618,0.412313,0.400255,0.364918,0.382609,0.416793,0.386992,0.370588,...,0.286136,0.363910,0.404121,0.376670,0.318328,0.273482,0.336515,0.403900,0.353448,0.402396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.459871,0.451993,0.383920,0.367969,0.454867,0.508562,0.545995,0.502857,0.513174,0.509506,...,0.425590,0.517221,0.435563,0.443043,0.280731,0.299484,0.300563,0.262145,0.261458,0.382929
2021-12-28,0.489694,0.459871,0.451993,0.383920,0.367969,0.454867,0.508562,0.545995,0.502857,0.513174,...,0.293302,0.425590,0.517221,0.435563,0.443043,0.280731,0.299484,0.300563,0.262145,0.261458
2021-12-29,0.429117,0.489694,0.459871,0.451993,0.383920,0.367969,0.454867,0.508562,0.545995,0.502857,...,0.255346,0.293302,0.425590,0.517221,0.435563,0.443043,0.280731,0.299484,0.300563,0.262145


In [6]:
df_mjo = pd.read_csv('./DATASET_COMPLETED/ensemble_dataset/MJO_30D_scaled.csv')
df_enso = pd.read_csv('./DATASET_COMPLETED/ensemble_dataset/ENSO_30D_scaled.csv')
df_target = pd.read_csv('./DATASET_COMPLETED/old_dataset/tc_act_sind.csv')

In [7]:
def rewind_att(att_list, steps):
  new_list = []
  for elem in att_list:
    for i in range(steps):
      new_list.append('{}-{}'.format(elem,i+1))
  return new_list

attributes_list = ['RMM1', 'RMM2', 'Phase', 'nino3', 'nino4', 'nino12', 'nino34', 'indocW','indocE']
#attributes_selected = ['RMM1', 'RMM2', 'Phase', 'indocW', 'indocE']
attributes_selected = ['RMM1', 'RMM2', 'Phase', 'nino3', 'nino4', 'nino12', 'nino34', 'indocW','indocE']

df_mjo_enso = pd.merge(df_mjo, df_enso, on=["DATE"])
df_mjo_enso = df_mjo_enso.set_index('DATE')
df_mjo_enso = df_mjo_enso[df_mjo_enso.index < '2022']
df_mjo_enso = df_mjo_enso[rewind_att(attributes_selected, 30)]

#print(len(df_mjo), len(df_enso), len(df_mjo_enso))
df_mjo_enso

,RMM1-1,RMM1-2,RMM1-3,RMM1-4,RMM1-5,RMM1-6,RMM1-7,RMM1-8,RMM1-9,RMM1-10,...,indocE-21,indocE-22,indocE-23,indocE-24,indocE-25,indocE-26,indocE-27,indocE-28,indocE-29,indocE-30
DATE,,,,,,,,,,,,,,,,,,,,,
1980-02-01,0.443395,0.497201,0.543049,0.533406,0.520222,0.496491,0.477807,0.479205,0.484783,0.492725,...,0.550539,0.539860,0.529143,0.519671,0.512739,0.512140,0.518311,0.527535,0.536084,0.540251
1980-02-02,0.426263,0.443395,0.497201,0.543049,0.533406,0.520222,0.496491,0.477807,0.479205,0.484783,...,0.559889,0.550539,0.539860,0.529143,0.519671,0.512739,0.512140,0.518311,0.527535,0.536084
1980-02-03,0.446730,0.426263,0.443395,0.497201,0.543049,0.533406,0.520222,0.496491,0.477807,0.479205,...,0.564371,0.559889,0.550539,0.539860,0.529143,0.519671,0.512739,0.512140,0.518311,0.527535
1980-02-04,0.477843,0.446730,0.426263,0.443395,0.497201,0.543049,0.533406,0.520222,0.496491,0.477807,...,0.562821,0.564371,0.559889,0.550539,0.539860,0.529143,0.519671,0.512739,0.512140,0.518311
1980-02-05,0.482835,0.477843,0.446730,0.426263,0.443395,0.497201,0.543049,0.533406,0.520222,0.496491,...,0.557513,0.562821,0.564371,0.559889,0.550539,0.539860,0.529143,0.519671,0.512739,0.512140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.456721,0.459083,0.487810,0.506327,0.522359,0.524730,0.508447,0.536899,0.574846,0.597357,...,0.585440,0.580454,0.595150,0.576959,0.601620,0.613510,0.615770,0.612231,0.609712,0.605784
2021-12-28,0.430960,0.456721,0.459083,0.487810,0.506327,0.522359,0.524730,0.508447,0.536899,0.574846,...,0.613950,0.585440,0.580454,0.595150,0.576959,0.601620,0.613510,0.615770,0.612231,0.609712
2021-12-29,0.412768,0.430960,0.456721,0.459083,0.487810,0.506327,0.522359,0.524730,0.508447,0.536899,...,0.606413,0.613950,0.585440,0.580454,0.595150,0.576959,0.601620,0.613510,0.615770,0.612231


In [8]:
df_target = df_target.rename(columns={'Unnamed: 0': 'DATE'})
df_target = df_target.set_index('DATE')
df_target['Label'] = df_target['S.IndAll'].apply(lambda x: 1 if x > 0 else 0)
df_target = df_target[df_target.index > '1980-02']
df_target = df_target[df_target.index < '2022']
df_target = df_target['Label']

df_target

DATE
1980-02-01    1
1980-02-02    1
1980-02-03    1
1980-02-04    1
1980-02-05    1
             ..
2021-12-27    0
2021-12-28    0
2021-12-29    0
2021-12-30    0
2021-12-31    0
Name: Label, Length: 15310, dtype: int64

## Build timeseries

In [9]:
def build_timeseries(df, N_samples, len_timeseries, N_vars):
  timeseries_arr = np.zeros((N_samples, len_timeseries, N_vars))
  arr = df.values
  i = 0
  for elem in arr:
    j = 0
    for var_idx in range(N_vars):
        idx = var_idx*len_timeseries
        timeseries_arr[i, :, j] = elem[idx : idx + len_timeseries]
        j += 1
    i += 1
  return timeseries_arr

valid_set = True

if valid_set:
  X_train_mjo = build_timeseries(df_mjo_enso[df_mjo_enso.index < '2002'], len(df_mjo_enso[df_mjo_enso.index < '2002']), 30, 5)
  X_train_era5 = build_timeseries(df_era5_scaled[df_era5_scaled.index < '2002'], len(df_era5_scaled[df_era5_scaled.index < '2002']), 11, 608)
  y_train = df_target[df_target.index < '2002'].values

  df_mjo_valid = df_mjo_enso[df_mjo_enso.index > '2002']
  df_mjo_valid = df_mjo_valid[df_mjo_valid.index < '2012']
  df_era5_valid = df_era5_scaled[df_era5_scaled.index > '2002']
  df_era5_valid = df_era5_valid[df_era5_valid.index < '2012']
  df_target_valid = df_target[df_target.index > '2002']
  df_target_valid = df_target_valid[df_target_valid.index < '2012']

  X_valid_mjo = build_timeseries(df_mjo_valid, len(df_mjo_valid), 30, 5)
  X_valid_era5 = build_timeseries(df_era5_valid, len(df_era5_valid), 11, 608)
  y_valid = df_target_valid.values

  X_test_mjo = build_timeseries(df_mjo_enso[df_mjo_enso.index > '2012'], len(df_mjo_enso[df_mjo_enso.index > '2012']), 30, 5)
  X_test_era5 = build_timeseries(df_era5_scaled[df_era5_scaled.index > '2012'], len(df_era5_scaled[df_era5_scaled.index > '2012']), 11, 608)
  y_test = df_target[df_target.index > '2012'].values
else:
  X_train_mjo = build_timeseries(df_mjo_enso[df_mjo_enso.index < '2012'], len(df_mjo_enso[df_mjo_enso.index < '2012']), 30, 5)
  X_train_era5 = build_timeseries(df_era5_scaled[df_era5_scaled.index < '2012'], len(df_era5_scaled[df_era5_scaled.index < '2012']), 11, 608)
  y_train = df_target[df_target.index < '2012'].values

  X_test_mjo = build_timeseries(df_mjo_enso[df_mjo_enso.index > '2012'], len(df_mjo_enso[df_mjo_enso.index > '2012']), 30, 5)
  X_test_era5 = build_timeseries(df_era5_scaled[df_era5_scaled.index > '2012'], len(df_era5_scaled[df_era5_scaled.index > '2012']), 11, 608)
  y_test = df_target[df_target.index > '2012'].values



print('TRAIN SET: ')
print('MJO    --> ', X_train_mjo.shape)
print('ERA5   --> ', X_train_era5.shape)
print('TARGET --> ', y_train.shape)

print('VALID SET: ')
print('MJO    --> ', X_valid_mjo.shape)
print('ERA5   --> ', X_valid_era5.shape)
print('TARGET --> ', y_valid.shape)

print('TEST SET: ')
print('MJO    --> ', X_test_mjo.shape)
print('ERA5   --> ', X_test_era5.shape)
print('TARGET --> ', y_test.shape)

TRAIN SET: 
MJO    -->  (8005, 30, 5)
ERA5   -->  (8005, 11, 608)
TARGET -->  (8005,)
VALID SET: 
MJO    -->  (3652, 30, 5)
ERA5   -->  (3652, 11, 608)
TARGET -->  (3652,)
TEST SET: 
MJO    -->  (3653, 30, 5)
ERA5   -->  (3653, 11, 608)
TARGET -->  (3653,)


In [10]:
8005+3652+3653

15310

In [11]:
X_train_future_mjo = []
X_train_future_era5 = []
X_valid_future_mjo = []
X_valid_future_era5 = []
X_test_future_mjo = []
X_test_future_era5 = []

y_train_future = []
y_valid_future = []
y_test_future = []

for i in range(10):
  if i == 0:
    X_train_future_mjo.append(X_train_mjo)
    X_train_future_era5.append(X_train_era5)

    X_valid_future_mjo.append(X_valid_mjo)
    X_valid_future_era5.append(X_valid_era5)

    X_test_future_mjo.append(X_test_mjo)
    X_test_future_era5.append(X_test_era5)

    y_train_future.append(y_train)
    y_valid_future.append(y_valid)
    y_test_future.append(y_test)
  else:
    X_train_future_mjo.append(X_train_mjo[:-i])
    X_train_future_era5.append(X_train_era5[:-i])

    X_valid_future_mjo.append(X_valid_mjo[:-i])
    X_valid_future_era5.append(X_valid_era5[:-i])

    X_test_future_mjo.append(X_test_mjo[:-i])
    X_test_future_era5.append(X_test_era5[:-i])

    y_train_future.append(y_train[i:])
    y_valid_future.append(y_valid[i:])
    y_test_future.append(y_test[i:])

  print('Time horizon: t+{}'.format(i+1))
  print('TRAIN SET: ')
  print('MJO    --> ', X_train_future_mjo[i].shape)
  print('ERA5   --> ', X_train_future_era5[i].shape)
  print('TARGET --> ', y_train_future[i].shape)

  print('VALID SET: ')
  print('MJO    --> ', X_valid_future_mjo[i].shape)
  print('ERA5   --> ', X_valid_future_era5[i].shape)
  print('TARGET --> ', y_valid_future[i].shape)

  print('TEST SET: ')
  print('MJO    --> ', X_test_future_mjo[i].shape)
  print('ERA5   --> ', X_test_future_era5[i].shape)
  print('TARGET --> ', y_test_future[i].shape)
  print('-------------------------------------------------')

Time horizon: t+1
TRAIN SET: 
MJO    -->  (8005, 30, 5)
ERA5   -->  (8005, 11, 608)
TARGET -->  (8005,)
VALID SET: 
MJO    -->  (3652, 30, 5)
ERA5   -->  (3652, 11, 608)
TARGET -->  (3652,)
TEST SET: 
MJO    -->  (3653, 30, 5)
ERA5   -->  (3653, 11, 608)
TARGET -->  (3653,)
-------------------------------------------------
Time horizon: t+2
TRAIN SET: 
MJO    -->  (8004, 30, 5)
ERA5   -->  (8004, 11, 608)
TARGET -->  (8004,)
VALID SET: 
MJO    -->  (3651, 30, 5)
ERA5   -->  (3651, 11, 608)
TARGET -->  (3651,)
TEST SET: 
MJO    -->  (3652, 30, 5)
ERA5   -->  (3652, 11, 608)
TARGET -->  (3652,)
-------------------------------------------------
Time horizon: t+3
TRAIN SET: 
MJO    -->  (8003, 30, 5)
ERA5   -->  (8003, 11, 608)
TARGET -->  (8003,)
VALID SET: 
MJO    -->  (3650, 30, 5)
ERA5   -->  (3650, 11, 608)
TARGET -->  (3650,)
TEST SET: 
MJO    -->  (3651, 30, 5)
ERA5   -->  (3651, 11, 608)
TARGET -->  (3651,)
-------------------------------------------------
Time horizon: t+4
TRAIN S

# LSTM Networks

## LSTM Network to classify MJO-ENSO + ERA5

In [12]:
# Classifier for the MJO-ENSO dataset
def build_LSTM_classifier(input_shape_mjo, input_shape_era5):
    # Build the autoencoder layer by layer
    input_layer_mjo = tfkl.Input(shape=input_shape_mjo, name='Input MJO-ENSO')
    input_layer_era5 = tfkl.Input(shape=input_shape_era5, name='Input ERA5')

    # LSTM layers - MJO_ENSO
    lstm_mjo = tfkl.LSTM(128, return_sequences=True, name='LSTM_1_MJO_ENSO')(input_layer_mjo)
    lstm_mjo = tfkl.LSTM(21, return_sequences=True, name='LSTM_2_MJO_ENSO')(lstm_mjo)
    lstm_mjo = tfkl.LSTM(21)(lstm_mjo)
    dropout_mjo = tfkl.Dropout(.5, seed=seed)(lstm_mjo)

    # LSTM layers - ERA5
    lstm_era5 = tfkl.LSTM(1025, return_sequences=True, name='LSTM_1_ERA5')(input_layer_era5)
    lstm_era5 = tfkl.LSTM(512, return_sequences=True, name='LSTM_2_ERA5')(lstm_era5)
    lstm_era5 = tfkl.LSTM(512)(lstm_era5)
    dropout_era5 = tfkl.Dropout(.5, seed=seed)(lstm_era5)

    # Dense Classifier
    concat = tfkl.Concatenate()([dropout_mjo, dropout_era5])
    classifier = tfkl.Dense(256, activation='relu')(concat)
    classifier = tfkl.Dense(64, activation='relu')(classifier)
    output_layer = tfkl.Dense(1, activation='sigmoid')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=[input_layer_mjo, input_layer_era5], outputs=output_layer, name='LSTM_Netwwork_MJO_ENSO_ERA5')
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [13]:
# Utility function to create folders and callbacks for training
from tensorflow.keras.callbacks import ModelCheckpoint
seed = 42

def create_callbacks(horizon) :
    callbacks = []

    # Early Stopping -----------------------------------------------------
    es_callback = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20, restore_best_weights=True)
    callbacks.append(es_callback)
    # Checkpointer -------------------------------------------------------
    checkpointer = ModelCheckpoint(filepath='./checkpoint/LSTM_MJO_ENSO_ERA5_t+{}.h5'.format(horizon), verbose=1,
                                    save_best_only=True, monitor = "val_loss", mode = "auto",)
    callbacks.append(checkpointer)
    # Learning Rate Scheduler --------------------------------------------
    LRS_callback = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    callbacks.append(LRS_callback)

    return callbacks

In [14]:
#Get a handle to the attached TPU. On GCP it will be the CloudTPU itself
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR']) # Connect to the TPU handle and initialise it
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [15]:
input_shape_mjo = X_train_mjo.shape[1:]
input_shape_era5 = X_train_era5.shape[1:]

classes = 2
batch_size = 128
epochs = 100
print('INPUT SHAPE: ', input_shape_mjo, input_shape_era5)
print('NUMBER OF CLASSES: ', classes)

INPUT SHAPE:  (30, 5) (11, 608)
NUMBER OF CLASSES:  2


In [16]:
from sklearn.utils import compute_class_weight

labels = df_target.values
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(labels),
                                        y = labels
                                    )
class_weights = dict(zip(np.unique(labels), class_weights))
print(class_weights)

def train_model(horizon):
  with strategy.scope():
    print('TRAINING MODEL AT TIME: t+{}'.format(horizon))
    model = build_LSTM_classifier(input_shape_mjo, input_shape_era5)
    #model.summary()
    history = model.fit(
        x = [X_train_future_mjo[horizon], X_train_future_era5[horizon]],
        y = y_train_future[horizon],
        epochs = epochs,
        validation_data = ([X_valid_future_mjo[horizon], X_valid_future_era5[horizon]], y_valid_future[horizon]),
        class_weight = class_weights,
        callbacks = create_callbacks(horizon)
    ).history

{0: 0.5828828142846265, 1: 3.5163068442811207}


In [18]:
# for i in range(10):
#   train_model(i)
train_model(8)

TRAINING MODEL AT TIME: t+8
Epoch 1/100
249/250 [============================>.] - ETA: 0s - loss: 0.7069 - accuracy: 0.4892
Epoch 1: val_loss improved from inf to 0.73143, saving model to ./checkpoint/LSTM_MJO_ENSO_ERA5_t+8.h5
250/250 [==============================] - 42s 74ms/step - loss: 0.7067 - accuracy: 0.4897 - val_loss: 0.7314 - val_accuracy: 0.1334 - lr: 0.0010
Epoch 2/100
249/250 [============================>.] - ETA: 0s - loss: 0.6999 - accuracy: 0.5245
Epoch 2: val_loss improved from 0.73143 to 0.65355, saving model to ./checkpoint/LSTM_MJO_ENSO_ERA5_t+8.h5
250/250 [==============================] - 13s 53ms/step - loss: 0.6990 - accuracy: 0.5257 - val_loss: 0.6535 - val_accuracy: 0.8666 - lr: 0.0010
Epoch 3/100
249/250 [============================>.] - ETA: 0s - loss: 0.6970 - accuracy: 0.5622
Epoch 3: val_loss improved from 0.65355 to 0.52181, saving model to ./checkpoint/LSTM_MJO_ENSO_ERA5_t+8.h5
250/250 [==============================] - 13s 53ms/step - loss: 0.6980 

In [ ]:
y_pred = model.predict([X_test_mjo, X_test_era5])
y_pred = y_pred.reshape(1, -1)[0]

print(y_pred.shape, y_test.shape)

In [ ]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score

pred = []
for elem in y_pred:
  if elem >= 0.5:
    pred.append(1)
  else:
    pred.append(0)
pred = np.array(pred)
print(y_pred)
print(pred)
print(y_test)

accuracy    = accuracy_score(y_test, pred)
recall      = recall_score(y_test, pred)
precision   = precision_score(y_test, pred)
f1          = f1_score(y_test, pred)


print('ACCURACY:  ', round(accuracy, 4))
print('RECALL:    ', round(recall, 4))
print('PRECISION: ', round(precision, 4))
print('F1_SCORE:  ', round(f1, 4))

[0.93657297 0.93936837 0.93774235 ... 0.01700619 0.0272707  0.23506013]
[1 1 1 ... 0 0 0]
[1 1 1 ... 0 0 0]
ACCURACY:   0.8965
RECALL:     0.7933
PRECISION:  0.5962
F1_SCORE:   0.6807
